# BIG DATA MIDDLE TERM EXAM
## Recommendation Systems 
- Name: Kurnia Cahya Febryanto
- Student ID: 5025201073
- Class: Big Data A
- Lecturer: Abdul Munif, S.Kom., M.Sc.

## Get File from Google Drive

In [2]:
# To be able to use your data stored in your Google Drive you first need to mount your Google Drive so you can load and save files to it. 
from google.colab import drive
drive.mount('/content/gdrive')
#You'll need to put in a token which Google will generate for you as soon as you click on the link

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pandas as pd
import csv

# Read the Excel file located in your Google Drive
data_file = '/content/gdrive/My Drive/BigData Source/market-basket.csv'

with open(data_file, 'r') as f:
    temp_lines = f.readline() + '\n' + f.readline()
    dialect = csv.Sniffer().sniff(temp_lines, delimiters=';,')
    f.seek(0)
    data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)
data.head()

<ipython-input-3-5a17a097eaaf>:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)
<ipython-input-3-5a17a097eaaf>:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f, dialect=dialect, error_bad_lines=False)


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


## Market Basket Analysis

In [4]:
%%capture
!sudo apt-get update --fix-missing

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
#!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

!mv spark-3.0.0-bin-hadoop3.2.tgz sparkkk
!tar xf sparkkk
!pip install -q findspark

In [5]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Sets up the environment variables for Java and Spark, initializes the findspark library
# Creates a new SparkSession for a PySpark application named 'fpgrowth'.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('fpgrowth') \
    .getOrCreate()

spark   

In [7]:
data = data.astype(str)

In [8]:
# Create Spark DataFrame from the pandas DataFrame
sparkdata = spark.createDataFrame(data)

/content/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:327: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [10]:
from pyspark.sql import functions as F

basketdata = sparkdata.dropDuplicates(['BillNo', 'Itemname']).sort('BillNo')
basketdata = basketdata.groupBy("BillNo").agg(F.collect_list("Itemname")).sort('BillNo')

### minSupport = 0.006 and minConfidence == 0.006

In [11]:
from pyspark.ml.fpm import FPGrowth

# Frequent Pattern Growth
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.006, minConfidence=0.006) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets

# Display generated association rules.
model.associationRules.show()
rules = model.associationRules

# Transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)


+--------------------+----+
|               items|freq|
+--------------------+----+
|[FELTCRAFT BUTTER...| 471|
|[FELTCRAFT BUTTER...| 167|
|[VINTAGE CHRISTMA...| 334|
|[ROCKING HORSE RE...| 245|
|[6 RIBBONS SHIMME...| 198|
|[VICTORIAN SEWING...| 160|
|[PARTY TIME PENCI...| 133|
|[WHITE HANGING HE...|2202|
|[STRAWBERRY FAIRY...| 198|
|[CAMOUFLAGE LED T...| 160|
|[PINK LOVE HEART ...| 133|
|[JUMBO BAG RED RE...|2064|
|[JUMBO BAG RED RE...| 442|
|[JUMBO BAG VINTAG...| 470|
|[JUMBO BAG VINTAG...| 136|
|[JUMBO BAG VINTAG...| 319|
|[JUMBO BAG VINTAG...| 130|
|[JUMBO BAG VINTAG...| 154|
|[JUMBO BAG VINTAG...| 180|
|[JUMBO BAG VINTAG...| 132|
+--------------------+----+
only showing top 20 rows

+--------------------+--------------------+-------------------+------------------+
|          antecedent|          consequent|         confidence|              lift|
+--------------------+--------------------+-------------------+------------------+
|[HOT WATER BOTTLE...|[LOVE HOT WATER B...| 0.3009079

In [12]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

,items,freq
0,[FELTCRAFT BUTTERFLY HEARTS],471
1,"[FELTCRAFT BUTTERFLY HEARTS, FELTCRAFT 6 FLOWE...",167
2,[VINTAGE CHRISTMAS STOCKING],334
3,[ROCKING HORSE RED CHRISTMAS],245
4,[6 RIBBONS SHIMMERING PINKS],198


In [13]:
result_pdf.to_excel('result_pdfFreq.xlsx')

In [14]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

,antecedent,consequent,confidence,lift
0,[HOT WATER BOTTLE KEEP CALM],[LOVE HOT WATER BOTTLE],0.300908,14.018426
1,[HOT WATER BOTTLE KEEP CALM],[CHOCOLATE HOT WATER BOTTLE],0.345006,8.940042
2,[HOT WATER BOTTLE KEEP CALM],[PAPER CHAIN KIT 50'S CHRISTMAS],0.216602,4.105201
3,[HOT WATER BOTTLE KEEP CALM],[GARDENERS KNEELING PAD KEEP CALM],0.225681,5.402128
4,[HOT WATER BOTTLE KEEP CALM],[SCOTTIE DOG HOT WATER BOTTLE],0.255512,7.292705


In [15]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift.xlsx')

In [16]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

,BillNo,collect_list(Itemname),prediction
0,536365,"[KNITTED UNION FLAG HOT WATER BOTTLE, SET 7 BA...","[JUMBO BAG RED RETROSPOT, REGENCY CAKESTAND 3 ..."
1,536366,"[HAND WARMER UNION JACK, HAND WARMER RED POLKA...","[HAND WARMER RED LOVE HEART, HAND WARMER RED R..."
2,536367,"[BOX OF VINTAGE JIGSAW BLOCKS, FELTCRAFT PRINC...","[POPPY'S PLAYHOUSE LIVINGROOM, POPPY'S PLAYHOU..."
3,536368,"[YELLOW COAT RACK PARIS FASHION, BLUE COAT RAC...","[JAM JAR WITH PINK LID, SMALL HEART MEASURING ..."
4,536369,[BATH BUILDING BLOCK WORD],[HOME BUILDING BLOCK WORD]


In [20]:
transformed_pdf.to_excel('transformed_pdfCollectList.xlsx')

### minSupport = 0.006 and minConfidence == 0.006

In [ ]:
from pyspark.ml.fpm import FPGrowth

# Frequent Pattern Growth
fpGrowth = FPGrowth(itemsCol="collect_list(Itemname)", minSupport=0.004, minConfidence=0.004) 
model = fpGrowth.fit(basketdata)

# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets

# Display generated association rules.
model.associationRules.show()
rules = model.associationRules

# Transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(basketdata).show()
transformed = model.transform(basketdata)


In [ ]:
# Convert the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = items.select("*").toPandas()
result_pdf.head()

In [ ]:
result_pdf.to_excel('result_pdfFreq2.xlsx')

In [ ]:
rules_pdf = rules.select("*").toPandas()
rules_pdf.head()

In [ ]:
rules_pdf.to_excel('rules_pdfAnteConseConfLift2.xlsx')

In [ ]:
transformed_pdf = transformed.select("*").toPandas()
transformed_pdf.head()

In [ ]:
transformed_pdf.to_excel('transformed_pdfCollectList2.xlsx')

## Summary

When comparing the two sets of parameters for a market basket analysis or association rule mining:

1. With minSupport = 0.006 and minConfidence = 0.006:

- The algorithm looks for itemsets that appear in at least 0.6% of the transactions.
- The generated association rules must have a confidence of at least 0.6% to be considered valid.

2. With minSupport = 0.004 and minConfidence = 0.004:

- The algorithm looks for itemsets that appear in at least 0.4% of the transactions.
- The generated association rules must have a confidence of at least 0.4% to be considered valid.

Comparing the two sets of parameters:
- Lowering the minSupport and minConfidence values from 0.006 to 0.004 will result in more itemsets and association rules being discovered, as the algorithm will consider less frequent and less confident rules.
- The output might include less relevant or less significant rules due to the lower thresholds.
- The computation time may increase, as the algorithm needs to process a larger number of itemsets and rules.